###  使用 官方的SQL链 实现 数据查询

LangChain 有一个 SQL Agent，它提供了一种比链式操作更灵活的与 SQL 数据库交互的方式。使用 SQL Agent 的主要优势包括：

它可以根据数据库的模式以及数据库的内容（比如描述一个特定的表）来回答问题。
它可以通过运行生成的查询，捕获追踪栈并正确地重新生成，来从错误中恢复。
它可以根据需要多次查询数据库以回答用户的问题。
它仅从相关表中检索的方式来节省令牌。
为了初始化agent ，我们将使用 SQLDatabaseToolkit 来创建一系列工具：

创建和执行查询
检查查询语法
检索表描述

In [2]:
from langchain.chat_models import  ChatOpenAI
from langchain.agents import initialize_agent,AgentType, Tool
from langchain.utilities import SQLDatabase
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, List, Optional, Any
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 获取 环境变量
#大模型apikey和url
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

#天气查询的apikey
juheAppKey = os.getenv('juheAppKey')

#数据库信息
mysql_info = os.getenv('mysql_info')

In [4]:
#定义大模型
online_model = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_37442/113483475.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  online_model = ChatOpenAI(


In [5]:
#定义数据库 连接
db = SQLDatabase.from_uri(mysql_info)


In [6]:
#查看数据库中的可用表
db.get_usable_table_names()

['city_stats']

In [7]:
#查看表结构
print(db.get_table_info())


CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/


In [8]:
#查看数据库信息
print(db.get_context())

{'table_info': '\nCREATE TABLE city_stats (\n\tcity_name VARCHAR(255), \n\tpopulation VARCHAR(255), \n\tarea VARCHAR(255)\n)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB\n\n/*\n3 rows from city_stats table:\ncity_name\tpopulation\tarea\n北京\t20000000\t400000234\n上海\t17689300\t29867200\n河北\t19999944\t39999821\n*/', 'table_names': 'city_stats'}


In [19]:
#使用官方 sql 数据库工具包
from langchain_community.agent_toolkits import SQLDatabaseToolkit

tookit = SQLDatabaseToolkit(db=db, llm=online_model)

#查看工具包中的工具
tools = tookit.get_tools()
tools



[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11e1a9ad0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11e1a9ad0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11e1a9ad0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [20]:
#定义提示词
from langchain_core.messages import SystemMessage
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""
system_message = SystemMessage(content=SQL_PREFIX)

In [26]:
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate

agent = initialize_agent(
    llm=online_model,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    system_message=system_message,
    verbose=True
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_37442/1158809702.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [27]:
agent.invoke('数据库中有哪些表？')



> Entering new AgentExecutor chain...
 需要查询数据库中的表列表。
Action: sql_db_list_tables
Action Input: 
Observation: city_stats
Thought:我现已知道数据库中包含的表。
Final Answer: 数据库中有一个名为 "city_stats" 的表。

> Finished chain.


{'input': '数据库中有哪些表？', 'output': '数据库中有一个名为 "city_stats" 的表。'}

In [28]:
agent.invoke('列出表中所有数据')



> Entering new AgentExecutor chain...
 我需要知道数据库中有哪些表，才能列出所有数据。
Action: sql_db_list_tables
Action Input: 
Observation: city_stats
Thought:我现在知道数据库中有一个名为 "city_stats" 的表。接下来，我需要查询该表的结构以了解其字段。
Action: sql_db_schema
Action Input: city_stats
Observation: 
CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/
Thought:我现在知道 "city_stats" 表的结构及其字段。接下来，我需要编写一个 SQL 查询以检索表中的所有数据，并使用 sql_db_query_checker 检查查询是否正确。
Action: sql_db_query_checker
Action Input: SELECT * FROM city_stats
Observation: SELECT * FROM city_stats
Thought:查询看起来是正确的。现在我将执行此查询以获取表中的所有数据。
Action: sql_db_query
Action Input: SELECT * FROM city_stats
Observation: [('北京', '20000000', '400000234'), ('上海', '17689300', '29867200'), ('河北', '19999944', '39999821')]
Thought:我现已获取到 "city_stats" 表中的所有数据。
Fin

{'input': '列出表中所有数据',
 'output': '城市名   人口       面积\n北京    20000000   400000234\n上海    17689300   29867200\n河北    19999944   39999821'}

In [29]:
agent.invoke('列出表中所有关于北京的数据')



> Entering new AgentExecutor chain...
我需要查询包含“北京”相关数据的表，但首先我需要知道数据库中有哪些表以及这些表的结构。
Action: sql_db_list_tables
Action Input: 
Observation: city_stats
Thought:我知道数据库中有一个名为 `city_stats` 的表。现在我需要查看该表的结构以了解其中包含哪些字段。
Action: sql_db_schema
Action Input: city_stats
Observation: 
CREATE TABLE city_stats (
	city_name VARCHAR(255), 
	population VARCHAR(255), 
	area VARCHAR(255)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from city_stats table:
city_name	population	area
北京	20000000	400000234
上海	17689300	29867200
河北	19999944	39999821
*/
Thought:我现在知道了 `city_stats` 表的结构，它包含以下字段：`city_name`, `population`, 和 `area`。并且我看到了一些样本数据，其中有一行与“北京”相关。现在我需要编写一个 SQL 查询来获取所有关于“北京”的数据。
Action: sql_db_query_checker
Action Input: SELECT * FROM city_stats WHERE city_name = '北京'
Observation: SELECT * FROM city_stats WHERE city_name = '北京'
Thought:我的查询语法正确，现在可以执行查询以获取结果。
Action: sql_db_query
Action Input: SELECT * FROM city_stats WHERE city_name = '北京'
Observation: [('北京', '20000000', '

{'input': '列出表中所有关于北京的数据',
 'output': '关于“北京”的数据如下：  \n城市名称：北京，人口：20000000，面积：400000234。'}